<a href="https://colab.research.google.com/github/mcgmed/Tensorflow/blob/main/Detecting-Features-in-Images/Horse_or_Human_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import zipfile

training_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(training_url, training_file_name)

zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

In [2]:
validation_url = "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        training_dir,
        target_size=(300, 300),
        class_mode='binary')

validation_datagen = ImageDataGenerator(rescale=1/255)

validation_generator = train_datagen.flow_from_directory(
        validation_dir,
        target_size=(300, 300),
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300,300,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 298, 298, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 149, 149, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 147, 147, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 73, 73, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 71, 71, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 35, 35, 64)       0

In [5]:
history = model.fit(train_generator, epochs=15, validation_data=validation_generator)

Epoch 1/15
33/33 [==============================] - 38s 828ms/step - loss: 0.7155 - accuracy: 0.5910 - val_loss: 0.7468 - val_accuracy: 0.5039
Epoch 2/15
33/33 [==============================] - 28s 844ms/step - loss: 0.5398 - accuracy: 0.7274 - val_loss: 1.7227 - val_accuracy: 0.5195
Epoch 3/15
33/33 [==============================] - 28s 840ms/step - loss: 0.4884 - accuracy: 0.7702 - val_loss: 1.4858 - val_accuracy: 0.4961
Epoch 4/15
33/33 [==============================] - 27s 818ms/step - loss: 0.4270 - accuracy: 0.8121 - val_loss: 4.6920 - val_accuracy: 0.5039
Epoch 5/15
33/33 [==============================] - 28s 841ms/step - loss: 0.3906 - accuracy: 0.8374 - val_loss: 1.9590 - val_accuracy: 0.5742
Epoch 6/15
33/33 [==============================] - 27s 813ms/step - loss: 0.3023 - accuracy: 0.8754 - val_loss: 1.5313 - val_accuracy: 0.6172
Epoch 7/15
33/33 [==============================] - 29s 867ms/step - loss: 0.2631 - accuracy: 0.8978 - val_loss: 2.3648 - val_accuracy: 0.5703

### **Testing**

In [6]:
import numpy as np
from google.colab import files
from tensorflow.keras.preprocessing import image

uploaded = files.upload()

Saving 0x0-horses-the-wings-of-mankind-1527015927739.jpg to 0x0-horses-the-wings-of-mankind-1527015927739.jpg
Saving gelding-bay-coat.webp to gelding-bay-coat.webp
Saving istockphoto-1365223878-170667a.jpg to istockphoto-1365223878-170667a.jpg


In [7]:
uploaded.keys()

dict_keys(['0x0-horses-the-wings-of-mankind-1527015927739.jpg', 'gelding-bay-coat.webp', 'istockphoto-1365223878-170667a.jpg'])

In [8]:
for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  image_tensor = np.vstack([x])
  classes = model.predict(image_tensor)
  print(classes)
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")

1/1 [==============================] - 0s 269ms/step
[[1.]]
0x0-horses-the-wings-of-mankind-1527015927739.jpg is a human
1/1 [==============================] - 0s 18ms/step
[[0.]]
gelding-bay-coat.webp is a horse
1/1 [==============================] - 0s 17ms/step
[[1.]]
istockphoto-1365223878-170667a.jpg is a human
